In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
pd.plotting.register_matplotlib_converters()
%matplotlib inline
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df=pd.read_csv("/kaggle/input/titanic/train.csv")
test_df=pd.read_csv("/kaggle/input/titanic/test.csv")
gender_s=pd.read_csv("/kaggle/input/titanic/gender_submission.csv")


Top 5 rows

In [ ]:
train_df.info()

In [ ]:
train_df.corr()

In [ ]:
#male selected
#train_df=train_df[train_df['Sex']=='male'].reset_index()
#test_df=test_df[test_df['Sex']=='male'].reset_index()

In [ ]:
train_df['family_count']=train_df['Parch']+train_df['SibSp']

test_df['family_count']=test_df['Parch']+test_df['SibSp']

# PreProcess

Handling Missing Values

In [ ]:
#Adding Is_Cabin
train_df['Is_Cabin']=train_df['Cabin'].notnull().astype(int)
X_C=train_df.groupby('Is_Cabin').mean().reset_index()
sns.barplot(x=X_C['Is_Cabin'],y=X_C['Survived'])

test_df['Is_Cabin']=test_df['Cabin'].notnull().astype(int)

In [ ]:
train_df['Cabin'].unique()

In [ ]:
train_df.columns

In [ ]:
#taking care of train_age missing
NaN_indexes = train_df['Age'][train_df['Age'].isnull()].index


for i in NaN_indexes:
    pred_age = train_df['Age'][((train_df.SibSp == train_df.iloc[i]["SibSp"]) & (train_df.Parch == train_df.iloc[i]["Parch"]) & (train_df.Pclass == train_df.iloc[i]["Pclass"]))].median()
    if not np.isnan(pred_age):
        train_df['Age'].iloc[i] = pred_age
    else:
        train_df['Age'].iloc[i] = train_df['Age'].median()

In [ ]:
#filling null with 'N/A'
train_df['Cabin'].fillna('N/A',inplace=True)
test_df['Cabin'].fillna('N/A',inplace=True)

In [ ]:
#Function to create Hall column using Cabin 
import re 
def Cabin_to_Hall(Cabin):
    hall = "".join(re.findall("[a-zA-Z]+",Cabin)) 
    return hall


In [ ]:
train_df['Hall']=train_df['Cabin'].apply(Cabin_to_Hall)
test_df['Hall']=test_df['Cabin'].apply(Cabin_to_Hall)

In [ ]:
def combine_hall(Hall):
    return  ''.join(set(Hall))

In [ ]:
train_df['Hall']=train_df['Hall'].apply(combine_hall)
test_df['Hall']=test_df['Hall'].apply(combine_hall)

In [ ]:
train_df['Hall'].unique()

In [ ]:
#from sklearn.preprocessing import OneHotEncoder
#
## Apply one-hot encoder to each column with categorical data
#OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
#OH_cols_train= pd.DataFrame(OH_encoder.fit_transform(pd.DataFrame(train_df['Hall'])))
#OH_cols_test = pd.DataFrame(OH_encoder.transform(pd.DataFrame(test_df['Hall'])))

In [ ]:
#getting original labels
#old_features=[]
#OH_features=list(OH_encoder.get_feature_names())
#for i in range(0,len(OH_features)):
#     old_features.append(OH_features[i].split("_")[1])

In [ ]:
#OH_cols_train

In [ ]:
#OH_cols_train.columns

In [ ]:
 #      #getting old names back
 #      OH_cols_train.columns = old_features  # initila train dataset for EDA
 #      OH_cols_test.columns = old_features  # for test
 #      
 #      
 #      #getting oringinal indexes back
 #      OH_cols_train.index = train_df.index
 #      OH_cols_test.index = test_df.index
 #      
 #      #Concatenate One Hot columns with original dataframes
 #      train_df = pd.concat([train_df, OH_cols_train], axis=1)
 #      test_df= pd.concat([test_df, OH_cols_test], axis=1)

In [ ]:
OH_cols_train.astye(int)

In [ ]:
train_df['Hall'].value_counts()


In [ ]:
pass_by_hall=train_df.groupby('Hall').mean().reset_index()
sns.barplot(x=pass_by_hall['Hall'],y=pass_by_hall['Survived'])

In [ ]:
#train_test_split
y=train_df.Survived
X=train_df.drop(['Survived'], axis=1)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,random_state=0)

In [ ]:
#Columns with missing values
cols=X_train.columns
for col in cols:
    if(X_train[col].isnull().sum()>0):
      print(col+"="+ str(X_train[col].isnull().sum()))

In [ ]:
#Putting median in place of Age
#age_median=X_train['Age'].median()
#X_train['Age']=X_train['Age'].fillna(age_median)
#X_valid['Age']=X_valid['Age'].fillna(age_median)

In [ ]:
#filling most common Embarked value
X_train['Embarked']=X_train['Embarked'].fillna('S')
X_valid['Embarked']=X_valid['Embarked'].fillna('S')

*Taking care of Catogorical Values*

In [ ]:
#def family_support(family_count):
#    if(family_count==3):
#        return 50
#    if(family_count==2):
#        return 40
#    if(family_count==6):
#        return 25
#    if(family_count==1):
#        return 23
#    if(family_count==0):
#        return 15
#    else: 
#        return 0

In [ ]:
def gender_to_num(Sex):
    if(Sex=='male'):
        return 0
    else:
        return 1

In [ ]:
def embarked_to_num(Emb):
    if(Emb=='C'):
        return 2;
    if(Emb=='Q'):
        return 1;
    else:
        return 0;

In [ ]:
def Pclass_to_distant(Pclass):
    if(Pclass==1):
        return 1
    if(Pclass==2):
        return 0
    if(Pclass==3):
        return -1

In [ ]:
#applying catogrical to numericalfunct
X_train['Sex'] = X_train['Sex'].apply(gender_to_num)
X_train['Embarked']=X_train['Embarked'].apply(embarked_to_num)
X_train['Pclass']=X_train['Pclass'].apply(Pclass_to_distant)

#X_train['family_support']=X_train['family_count'].apply(family_support)

X_valid['Sex'] = X_valid['Sex'].apply(gender_to_num)
X_valid['Embarked']=X_valid['Embarked'].apply(embarked_to_num)
X_valid['Pclass']=X_valid['Pclass'].apply(Pclass_to_distant)

#X_valid['family_support']=X_valid['family_count'].apply(family_support)


test_df['Sex'] = test_df['Sex'].apply(gender_to_num)
test_df['Embarked']=test_df['Embarked'].apply(embarked_to_num)
test_df['Pclass']=test_df['Pclass'].apply(Pclass_to_distant)

#test_df['family_support']=test_df['family_count'].apply(family_support)


In [ ]:
print(np.sort(X_train['Hall'].unique()))
print(np.sort(test_df['Hall'].unique()))

In [ ]:
Oh_cols_train_df

In [ ]:
from sklearn.preprocessing import LabelEncoder


label_encoder = LabelEncoder()

label_encoder.fit(pd.concat([train_df['Hall'], test_df['Hall']], axis=0, sort=False))

X_train['Hall'] = label_encoder.transform(X_train['Hall'])
X_valid['Hall'] = label_encoder.transform(X_valid['Hall'])  
test_df['Hall'] = label_encoder.transform(test_df['Hall'])

In [ ]:
#Dropping Cabin
X_train=X_train.drop(['Cabin'], axis=1)
X_valid=X_valid.drop(['Cabin'], axis=1)

In [ ]:
test_df.info()

# EDA

In [ ]:
train_df.head()

In [ ]:
male_df=train_df[train_df['Sex']=='male']
male_df['Age_tranch']=pd.cut(male_df['Age'], bins=[0,12,20,30,40,50,120], labels=['Children','Teenage','Adult20s','Adult30s','Adult_Elder','Elder'])
male_age=male_df.groupby('Age_tranch').mean().reset_index()
sns.barplot(x=male_age['Age_tranch'],y=male_age['Survived'])

In [ ]:
x_c=train_df.groupby('family_count').mean().reset_index()
sns.barplot(x=x_c['family_count'],y=x_c['Survived'])
male_df[male_df['family_count']==3]



In [ ]:
train_df['family_count']=train['Parch']+male_df['SibSp']
x_c=male_df.groupby('family_count').mean().reset_index()
sns.barplot(x=x_c['family_count'],y=x_c['Survived'])
male_df[male_df['family_count']==3]

In [ ]:
train_df['Sex'].value_counts()

In [ ]:
 train_df['Age_tranch']=pd.cut(train_df['Age'], bins=[0,12,20,40,50,120], labels=['Children','Teenage','Adult','Early Adult','Elder'])

#  Age vs Survived

In [ ]:
count_Embarked=train_df.groupby('Embarked').mean().reset_index()
sns.barplot(x=count_Embarked['Embarked'],y=count_Embarked['Survived'])

In [ ]:
age_survived=train_df[train_df['Sex']=='male'].groupby('Age_tranch').mean().reset_index()
age_survived=age_survived[['Age_tranch','Survived']]


In [ ]:
age_survived=train_df[train_df['Sex']=='female'].groupby('Age_tranch').mean().reset_index()
age_survived=age_survived[['Age_tranch','Survived']]
sns.barplot(x=age_survived['Age_tranch'],y=age_survived['Survived'])

In [ ]:
sns.catplot('Age_tranch', col = 'Survived', data = train_df, kind = 'count')


In [ ]:
train_df.info()

In [ ]:
test_df

In [ ]:
train_df[['Cabin','Survived']]

In [ ]:
X_train.info()

In [ ]:
train_df.Age.max()

In [ ]:
x_c=train_df[['Fare','Pclass']]

# **Preprocessing test dataset **

In [ ]:
test_df

In [ ]:
#missing values in test_df
cols=test_df.columns
for col in cols:
    if(test_df[col].isnull().sum()>0):
      print(col+"="+ str(test_df[col].isnull().sum()))

In [ ]:
#taking care of test_df AGE
NaN_indexes = test_df['Age'][test_df['Age'].isnull()].index
for i in NaN_indexes:
    pred_age = test_df['Age'][((test_df.SibSp == test_df.iloc[i]["SibSp"]) & (test_df.Parch == test_df.iloc[i]["Parch"]) & (test_df.Pclass == test_df.iloc[i]["Pclass"]))].median()
    if not np.isnan(pred_age):
        test_df['Age'].iloc[i] = pred_age
    else:
        test_df['Age'].iloc[i] = test_df['Age'].median()

In [ ]:
test_df.info()

In [ ]:
#handling missing values in test_df
#test_df['Age']=test_df['Age'].fillna(age_median)

fare_median=test_df['Fare'].median()
print(fare_median)
test_df['Fare']=test_df['Fare'].fillna(fare_median)

In [ ]:
train_df.info()
#train_df.groupby('Cabin').mean().reset_index()

In [ ]:
#NaN_indexes = test_df['Pclass'][test_df['Pclass'].isnull()].index

#for i in NaN_indexes:
 #    print(test_df.iloc[i]['Fare']) 
   #    
   #    if(test_df.iloc[i]['Fare']>=26):
   #         test_df.iloc[i]['Pclass']=1
   #    if(test_df.iloc[i]['Fare']<26 and test_df.iloc[i]['Fare']>=16):
   #        test_df.iloc[i]['Pclass']= 0
   #    if(test_df.iloc[i]['Fare']<16):
   #        test_df.iloc[i]['Pclass'] =-1

In [ ]:
test_df.head()

# # # Modelling

In [ ]:
train_df[train_df['Fare']==0]

In [ ]:
pd.concat([X_train,y_train],axis=1).corr()

In [ ]:
train_df[train_df['Cabin'].notnull()].Survived.mean()
train_df.groupby('Cabin').count().reset_index().max()

In [ ]:
Features=['Sex','Pclass','Age','Fare','Parch','SibSp', 'Embarked','Is_Cabin','Hall']
final_X_train=X_train[Features]
final_X_valid=X_valid[Features]
final_test=test_df[Features]


In [ ]:
final_test.info()

In [ ]:
final_test.info()

# **MODEL FITTING**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

n_logreg = LogisticRegression()
n_logreg.fit(final_X_train,y_train)
print('Test accuracy score:', accuracy_score(y_valid,n_logreg.predict(final_X_valid)))

In [ ]:
logi_pred=n_logreg.predict(final_test)

In [ ]:
submission_logistic = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": logi_pred})

In [ ]:
submission_logistic.to_csv('submission_logistic.csv', index=False)
print("Your submission was successfully saved!")

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

l_titanic_model = RandomForestClassifier()

# fit your model
l_titanic_model.fit(final_X_train,y_train)
for n_est in range(10,500,10):
             scores = -1 * cross_val_score( RandomForestClassifier(n_estimators=n_est), final_X_train, y_train,
                              cv=5,
                              scoring='neg_mean_absolute_error')
             print(str(n_est)+"= "+str(scores.mean()))

# Calculate the mean absolute error of your Random Forest model on the validation data
#rf_val_mae = mean_absolute_error(rf_model.predict(val_X),val_y)


In [ ]:
random_titanic = RandomForestClassifier(n_estimators=40,random_state=0)

# fit your model
random_titanic.fit(final_X_train,y_train)

In [ ]:
predicted_value=random_titanic.predict(final_X_valid)


In [ ]:
mean_absolute_error(predicted_value,y_valid)

In [ ]:
final_test.info()

In [ ]:
predicted_value=random_titanic.predict(final_test)


In [ ]:
submission_RF = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": predicted_value})

In [ ]:
submission_RF.to_csv('submission_final.csv', index=False)
print("Your submission was successfully saved!")

In [ ]:
my_model = XGBRegressor(n_estimators=1000, learning_rate=0.01, n_jobs=20)
my_model.fit(final_X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(final_X_valid, y_valid)], 
             verbose=False)

In [ ]:
predicted_value=my_model.predict(final_X_valid)


In [ ]:
mean_absolute_error(predicted_value,y_valid)

In [ ]:
submission_female = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": female_test})

In [ ]:
submission

In [ ]:
submission_female.to_csv('my_submission_female.csv', index=False)
print("Your submission was successfully saved!")

In [ ]:
submission_male=pd.read_csv('submission_male.csv')
submission_female=pd.read_csv('submission_female.csv')

In [ ]:
submission_final= pd.concazt([submission_male, submission_female])
submission_final.to_csv('my_submission_final.csv', index=False)
print("Your submission was successfully saved!")

In [ ]:
submission_final.info()